In [1]:
import numpy as np
import pandas as pd
import re
from pprint import pprint
qc_raw = pd.read_excel("zfssz3.result.xlsx", sheet_name="QC信息表")
seq_raw = pd.read_excel("sequence.20200428.xlsx", sheet_name="测序信息表")
pooling_raw = pd.read_excel("pooling.20200428.xlsx", sheet_name="pooling")
capture_raw = pd.read_excel("pooling.20200428.xlsx", sheet_name="capture")
library_raw = pd.read_excel("library.20200424.v2.xlsx", sheet_name=0)
extract_raw = pd.read_excel("extract.20200430.xlsx", sheet_name="merge")
clinical_raw = pd.read_excel("clinical.20200430.xlsx", sheet_name="merge")
sampleId_unique_overlap = pd.read_excel("library.20200424.v2.xlsx", sheet_name=1).loc[:,"样本编号"]

In [2]:
clinical_raw.head

<bound method NDFrame.head of             样本编号   姓名   性别    年龄      住院号     癌种   分期          诊断  \
0     19P3758198  李春萱    女  57.0  19P0501    NaN  NaN      BGI健康人   
1     19P3758199   蔡流    男  60.0  19P0502    NaN  NaN      BGI健康人   
2     19P3758200   钟燕    女  57.0  19P0503    NaN  NaN      BGI健康人   
3     19P3758201  彭小中    女  51.0  19P0504    NaN  NaN      BGI健康人   
4     19P3758202  韩桂芬    女  60.0  19P0505    NaN  NaN      BGI健康人   
...          ...  ...  ...   ...      ...    ...  ...         ...   
1033  19P5803657  杜亚洲    男  49.0  1037698  肝癌MRD    A  肝右叶原发性肝癌     
1034  19P5803658  叶子才    男  65.0  1039390  肝癌MRD    A    肝占位性病变     
1035  19P5803659  孙堂进    男  75.0  1039058  肝癌MRD    A     原发性肝癌     
1036  19P5803660  陈星伟    男  75.0  1038419  肝癌MRD    A     原发性肝癌     
1037  19P6714067  周竟成  NaN   NaN  1010495  肝癌MRD  NaN         NaN   

                                                   诊断备注       采样日期     医院编号  \
0                                    肿瘤患病史: 无, 家族肿瘤史: 无 2019-0

In [3]:
clinical_raw_filt = clinical_raw.loc[clinical_raw.loc[:, "样本编号"].isin(sampleId_unique_overlap),:]
clinical_raw_filt.head


<bound method NDFrame.head of             样本编号   姓名   性别    年龄      住院号     癌种   分期          诊断  \
0     19P3758198  李春萱    女  57.0  19P0501    NaN  NaN      BGI健康人   
1     19P3758199   蔡流    男  60.0  19P0502    NaN  NaN      BGI健康人   
2     19P3758200   钟燕    女  57.0  19P0503    NaN  NaN      BGI健康人   
3     19P3758201  彭小中    女  51.0  19P0504    NaN  NaN      BGI健康人   
4     19P3758202  韩桂芬    女  60.0  19P0505    NaN  NaN      BGI健康人   
...          ...  ...  ...   ...      ...    ...  ...         ...   
1033  19P5803657  杜亚洲    男  49.0  1037698  肝癌MRD    A  肝右叶原发性肝癌     
1034  19P5803658  叶子才    男  65.0  1039390  肝癌MRD    A    肝占位性病变     
1035  19P5803659  孙堂进    男  75.0  1039058  肝癌MRD    A     原发性肝癌     
1036  19P5803660  陈星伟    男  75.0  1038419  肝癌MRD    A     原发性肝癌     
1037  19P6714067  周竟成  NaN   NaN  1010495  肝癌MRD  NaN         NaN   

                                                   诊断备注       采样日期     医院编号  \
0                                    肿瘤患病史: 无, 家族肿瘤史: 无 2019-0

In [4]:
extract_raw.head

<bound method NDFrame.head of            DNA提取编号        样本编号       提取日期 样本类型 样本体积     提取方法     浓度    体积  备注
0     19P3758198_1  19P3758198 2019-04-24   血浆    9      NaN  0.608  73.0 NaN
1     19P3758199_1  19P3758199 2019-04-24   血浆    5      NaN  0.928  42.0 NaN
2     19P3758200_1  19P3758200 2019-04-24   血浆  5.5      NaN  0.772  42.0 NaN
3     19P3758201_1  19P3758201 2019-04-24   血浆  9.5      NaN  0.540  73.0 NaN
4     19P3758202_1  19P3758202 2019-04-24   血浆  9.1      NaN  0.470  73.0 NaN
...            ...         ...        ...  ...  ...      ...    ...   ...  ..
1020  19P5803657_1  19P5803657        NaT   血浆    4  美基XP双选法  6.140  43.0 NaN
1021  19P5803655_1  19P5803655        NaT   血浆    4  美基XP双选法  8.140  43.0 NaN
1022  19P5803660_1  19P5803660        NaT   血浆    4  美基XP双选法  1.530  43.0 NaN
1023  19P5803659_1  19P5803659        NaT   血浆    4  美基XP双选法  4.060  43.0 NaN
1024  19P5803658_1  19P5803658        NaT   血浆    4  美基XP双选法  8.600  43.0 NaN

[1025 rows x 9 columns]>

In [5]:
extract_raw_filt = extract_raw.loc[extract_raw.loc[:, "样本编号"].isin(sampleId_unique_overlap), :]
extract_raw_filt.head

<bound method NDFrame.head of            DNA提取编号        样本编号       提取日期 样本类型 样本体积     提取方法     浓度    体积  备注
0     19P3758198_1  19P3758198 2019-04-24   血浆    9      NaN  0.608  73.0 NaN
1     19P3758199_1  19P3758199 2019-04-24   血浆    5      NaN  0.928  42.0 NaN
2     19P3758200_1  19P3758200 2019-04-24   血浆  5.5      NaN  0.772  42.0 NaN
3     19P3758201_1  19P3758201 2019-04-24   血浆  9.5      NaN  0.540  73.0 NaN
4     19P3758202_1  19P3758202 2019-04-24   血浆  9.1      NaN  0.470  73.0 NaN
...            ...         ...        ...  ...  ...      ...    ...   ...  ..
1020  19P5803657_1  19P5803657        NaT   血浆    4  美基XP双选法  6.140  43.0 NaN
1021  19P5803655_1  19P5803655        NaT   血浆    4  美基XP双选法  8.140  43.0 NaN
1022  19P5803660_1  19P5803660        NaT   血浆    4  美基XP双选法  1.530  43.0 NaN
1023  19P5803659_1  19P5803659        NaT   血浆    4  美基XP双选法  4.060  43.0 NaN
1024  19P5803658_1  19P5803658        NaT   血浆    4  美基XP双选法  8.600  43.0 NaN

[1025 rows x 9 columns]>

In [6]:
library_raw.head

<bound method NDFrame.head of                  建库编号       DNA提取编号 管上编号    是否临床        文库名 样本标签 index列表  \
0     18S4146706_1_m1  18S4146706_1    1  Clinic      ssM-1    P   57,58   
1     18S4146707_1_m1  18S4146707_1    2  Clinic      ssM-2    P   59,60   
2     18S4146708_1_m1  18S4146708_1    3  Clinic      ssM-3    P   61,62   
3     18S4146709_1_m1  18S4146709_1    4  Clinic      ssM-4    P   63,64   
4     18S4146710_1_m1  18S4146710_1    5  Clinic      ssM-5    P   65,66   
...               ...           ...  ...     ...        ...  ...     ...   
1100  20L5383633_1_m1  20L5383633_1   12  Clinic  ssM-12-pl    T      68   
1101  20S5383594_1_m1  20S5383594_1   13  Clinic  ssM-13-pl    T      69   
1102  20L5383634_1_m1  20L5383634_1   14  Clinic  ssM-14-pl    T      70   
1103  20S5383595_1_m1  20S5383595_1   15  Clinic  ssM-15-pl    T      71   
1104  20L5383635_1_m1  20L5383635_1   16  Clinic  ssM-16-pl    T      72   

            建库日期  建库方法    试剂批次  ...                      

In [7]:
library_raw_filt = library_raw.loc[library_raw.loc[:, "DNA提取编号"].isin(extract_raw_filt.loc[:, "DNA提取编号"]), :]
library_raw_filt.head

<bound method NDFrame.head of                  建库编号       DNA提取编号 管上编号    是否临床        文库名 样本标签 index列表  \
10    18P4063823_1_m1  18P4063823_1    5  Clinic          5    P   65,66   
11    18P4063845_1_m1  18P4063845_1   16  Clinic         16    P   71,72   
12    18P4063830_1_m1  18P4063830_1   23  Clinic     ssM-23    P   67,68   
13    18P4063833_1_m1  18P4063833_1   31  Clinic     ssM-31    P   69,70   
14    18P4063839_1_m1  18P4063839_1   38  Clinic     ssM-38    P   63,64   
...               ...           ...  ...     ...        ...  ...     ...   
1060  19P7247863_1_m1  19P7247863_1   20  Clinic  ssM-20-pl    P      76   
1061  19P7247850_1_m1  19P7247850_1   21  Clinic  ssM-21-pl    P      77   
1062  19P7247862_1_m1  19P7247862_1   22  Clinic  ssM-22-pl    P      78   
1063  19P7247867_1_m1  19P7247867_1   23  Clinic  ssM-23-pl    P      79   
1064  19P7247884_1_m1  19P7247884_1   24  Clinic  ssM-24-pl    P      80   

            建库日期  建库方法    试剂批次  ...                      

In [12]:
count = {}
tmp = []
for idx in range(pooling_raw.shape[0]):
    k = pooling_raw.loc[idx, "建库编号"]
    count[k] = 1+count[k] if k in count else 1 
    tmp.append("{}_{}".format(k, count[k]))
pooling_raw.loc[:, "测序文库名"] = tmp


In [13]:
pooling_raw.head

<bound method NDFrame.head of                  建库编号                    文库名称  虚拟入库样本名  pooling比例  取样ng  \
0     18S4146706_1_m1  ssM-CpGiant-1-20181218  IDTL130        1.0   250   
1     18S4146707_1_m1  ssM-CpGiant-1-20181218  IDTL130        1.0   250   
2     18S4146708_1_m1  ssM-CpGiant-1-20181218  IDTL130        1.0   250   
3     18S4146709_1_m1  ssM-CpGiant-1-20181218  IDTL130        1.0   250   
4     18S4146710_1_m1  ssM-CpGiant-2-20181218  IDTL131        1.0   250   
...               ...                     ...      ...        ...   ...   
1078  20L5383633_1_m1  ssM-CpGiant-6-20200214    LM310        1.0   250   
1079  20S5383594_1_m1  ssM-CpGiant-6-20200214    LM310        1.0   250   
1080  20L5383634_1_m1  ssM-CpGiant-6-20200214    LM310        1.0   250   
1081  20S5383595_1_m1  ssM-CpGiant-6-20200214    LM310        1.0   250   
1082  20L5383635_1_m1  ssM-CpGiant-6-20200214    LM310        1.0   250   

           体积ul              测序文库名  
0      4.166667  18S4146706_1_m1

In [14]:
pooling_raw_filt = pooling_raw.loc[pooling_raw.loc[:, "建库编号"].isin(library_raw_filt.loc[:, "建库编号"]), :]
pooling_raw_filt.head

<bound method NDFrame.head of                  建库编号                    文库名称  虚拟入库样本名  pooling比例  取样ng  \
10    18P4063823_1_m1  ssM-CpGiant-4-20190104  IDTL145        1.0   500   
11    18P4063845_1_m1  ssM-CpGiant-4-20190104  IDTL145        1.0   500   
12    18P4063830_1_m1  ssM-CpGiant-2-20190109    LM001        1.0   500   
13    18P4063833_1_m1  ssM-CpGiant-3-20190109    LM002        1.0   500   
14    18P4063839_1_m1  ssM-CpGiant-4-20190109    LM002        1.0   500   
...               ...                     ...      ...        ...   ...   
1038  19P7247863_1_m1  ssM-CpGiant-8-20200131    LM289        1.0   250   
1039  19P7247850_1_m1  ssM-CpGiant-8-20200131    LM289        1.0   250   
1040  19P7247862_1_m1  ssM-CpGiant-8-20200131    LM289        1.0   250   
1041  19P7247867_1_m1  ssM-CpGiant-8-20200131    LM289        1.0   250   
1042  19P7247884_1_m1  ssM-CpGiant-8-20200131    LM289        1.0   250   

           体积ul              测序文库名  
10     9.505703  18P4063823_1_m1

In [15]:
# pooling_raw_filt.loc[:,"文库名称"].value_counts()

In [16]:
capture_raw.head

<bound method NDFrame.head of                        文库名称  虚拟入库样本名        杂交日期  \
0    ssM-CpGiant-1-20181218  IDTL130  2018/12/18   
1    ssM-CpGiant-2-20181218  IDTL131  2018/12/18   
2    ssM-CpGiant-1-20190104  IDTL142  2019/01/04   
3    ssM-CpGiant-4-20190104  IDTL145  2019/01/04   
4    ssM-CpGiant-2-20190109    LM001  2019/01/09   
..                      ...      ...         ...   
212  ssM-CpGiant-2-20200214    LM306  2020/02/14   
213  ssM-CpGiant-3-20200214    LM307  2020/02/14   
214  ssM-CpGiant-4-20200214    LM308  2020/02/14   
215  ssM-CpGiant-5-20200214    LM309  2020/02/14   
216  ssM-CpGiant-6-20200214    LM310  2020/02/14   

                                  杂交探针  杂交时间h  PostPCR循环数 PostPCR浓度ng/ul  \
0    SeqCap Epi CpGiant Enrichment Kit   66.5          12           16.2   
1    SeqCap Epi CpGiant Enrichment Kit   67.5          13           17.6   
2    SeqCap Epi CpGiant Enrichment Kit   67.0          12            110   
3    SeqCap Epi CpGiant Enrichment Kit   

In [17]:
capture_raw_filt = capture_raw.loc[capture_raw.loc[:, "文库名称"].isin(pooling_raw_filt.loc[:, "文库名称"]), :]
capture_raw_filt.head

<bound method NDFrame.head of                        文库名称  虚拟入库样本名        杂交日期  \
3    ssM-CpGiant-4-20190104  IDTL145  2019/01/04   
4    ssM-CpGiant-2-20190109    LM001  2019/01/09   
5    ssM-CpGiant-3-20190109    LM002  2019/01/09   
6    ssM-CpGiant-4-20190109    LM002  2019/01/09   
7    ssM-CpGiant-5-20190109    LM003  2019/01/09   
..                      ...      ...         ...   
207  ssM-CpGiant-6-20200131    LM287  2020/01/31   
208  ssM-CpGiant-7-20200131    LM287  2020/01/31   
209  ssM-CpGiant-6-20200131    LM319  2020/01/31   
210  ssM-CpGiant-7-20200131    LM319  2020/01/31   
211  ssM-CpGiant-8-20200131    LM289  2020/01/31   

                                  杂交探针  杂交时间h  PostPCR循环数 PostPCR浓度ng/ul  \
3    SeqCap Epi CpGiant Enrichment Kit   67.0          12            110   
4    SeqCap Epi CpGiant Enrichment Kit   70.0          11           38.8   
5    SeqCap Epi CpGiant Enrichment Kit   70.0          11           59.8   
6    SeqCap Epi CpGiant Enrichment Kit   

In [18]:
seq_raw.head

<bound method NDFrame.head of              虚拟入库样本名1        送测日期        上机时间        下机时间  \
0    IDTL130, IDTL131         NaN  2018/12/27  2018/12/29   
1             IDTL142         NaN  2019/01/12  2019/01/14   
2             IDTL145         NaN  2019/01/12  2019/01/14   
3               LM001         NaN  2019/01/16  2019/01/18   
4               LM002         NaN  2019/01/16  2019/01/19   
..                ...         ...         ...         ...   
149             LM284  2020/02/03  2020/02/04  2020/02/06   
150             LM285  2020/02/03  2020/02/04  2020/02/06   
151             LM286  2020/02/04  2020/02/05  2020/02/07   
152             LM287  2020/02/04  2020/02/05  2020/02/07   
153             LM289  2020/02/04  2020/02/05  2020/02/07   

                              机器号                     芯片号  \
0                   R100400180038  V300012489, V300012514   
1                   R100400180015  V300012566, V300012600   
2                   R100400180007  V300012612, V300012

In [19]:
filter = []
for idx in range(seq_raw.shape[0]):
    id_list = str(seq_raw.loc[idx, "捕获文库名"]).split(", ")
    match_mark = 0
    for id in id_list:
        id = id.strip()
        if id in capture_raw_filt.loc[:, "文库名称"].to_string().strip():
            match_mark = 1
            break
    if match_mark:
        filter.append(True)
    else:
        filter.append(False)
seq_raw_filt = seq_raw.loc[filter, :]
seq_raw_filt.head

<bound method NDFrame.head of          虚拟入库样本名1        送测日期        上机时间        下机时间  \
2         IDTL145         NaN  2019/01/12  2019/01/14   
3           LM001         NaN  2019/01/16  2019/01/18   
4           LM002         NaN  2019/01/16  2019/01/19   
5           LM004         NaN  2019/01/21  2019/01/23   
6    LM004, LM005         NaN  2019/01/26  2019/01/28   
..            ...         ...         ...         ...   
149         LM284  2020/02/03  2020/02/04  2020/02/06   
150         LM285  2020/02/03  2020/02/04  2020/02/06   
151         LM286  2020/02/04  2020/02/05  2020/02/07   
152         LM287  2020/02/04  2020/02/05  2020/02/07   
153         LM289  2020/02/04  2020/02/05  2020/02/07   

                              机器号                     芯片号  \
2                   R100400180007  V300012612, V300012618   
3                   R100400180015  V300012535, V300012554   
4                   R100400180004  V300012593, V300012614   
5                   R100400180004  V30001

In [21]:
count = {}
tmp = []
for idx in range(qc_raw.shape[0]):
    k = qc_raw.loc[idx, "sampleID_shift"]
    count[k] = 1+count[k] if k in count else 1
    tmp.append("{}_{}".format(k, count[k]))
qc_raw.loc[:, "测序文库名"] = tmp

In [22]:
qc_raw.head

<bound method NDFrame.head of            Sample  Data_Size(Gb) Clean_Rate(%)  R1_Q20  R2_Q20  R1_Q30  \
0     18S4146666M          40.23        93.45%  97.42%  94.71%  90.07%   
1     18S4146667M          37.36        93.36%  97.32%  94.48%  89.72%   
2     18S4146668M          30.61        93.32%  97.21%  94.59%  89.35%   
3     18S4146669M          32.03        93.65%  97.24%  94.85%  89.48%   
4     18S4070982W          36.96        92.09%  97.48%  94.92%  89.78%   
...           ...            ...           ...     ...     ...     ...   
1936  20L5383610M          36.98        96.23%  97.99%  96.77%  91.41%   
1937  20S5383649M          33.65        97.04%  97.99%  96.88%  91.39%   
1938  20L5383609M          33.50        96.73%  97.86%  96.64%  90.98%   
1939  20L5383614M          34.46        94.11%  97.40%  95.61%  89.58%   
1940  20S5383654M          36.87        94.39%  97.85%  97.17%  91.04%   

      R2_Q30 GC_Content BS_conversion_rate(lambda_DNA)  \
0     84.46%     30.91%

In [23]:
qc_raw_filt = qc_raw.loc[qc_raw.loc[:, "best_seqId"].isin(seq_raw_filt.loc[:,"测序编号"]), :]
qc_raw_filt.head

<bound method NDFrame.head of            Sample  Data_Size(Gb) Clean_Rate(%)  R1_Q20  R2_Q20  R1_Q30  \
20    18P4063775M          33.83        90.11%  97.72%  93.98%  90.63%   
24    18P4063823M          34.26        88.28%  97.83%  94.13%  91.06%   
25    18P4062832M          29.38        89.68%  97.60%  93.85%  90.19%   
27    18P4063819M          30.79        90.46%  97.62%  94.13%  90.35%   
28    18P4063845M          32.48        90.66%  97.61%  94.01%  90.27%   
...           ...            ...           ...     ...     ...     ...   
1848  19P7247843M          37.42        93.36%  97.73%  96.36%  90.51%   
1849  19P7247847M          44.69        95.63%  98.07%  97.10%  91.30%   
1850  19P7247853M          43.26        95.02%  97.77%  96.98%  90.31%   
1851  19P7247861M          37.82        94.15%  97.78%  96.56%  90.67%   
1852  19P7247863M          35.73        93.61%  97.61%  96.48%  90.15%   

      R2_Q30 GC_Content BS_conversion_rate(lambda_DNA)  \
20    83.42%     28.30%

In [33]:
qc_raw_filt_subset1 = qc_raw_filt.loc[qc_raw_filt.loc[:, "样本编号"].isin(sampleId_unique_overlap), :]
qc_raw_filt_subset1.head

<bound method NDFrame.head of            Sample  Data_Size(Gb) Clean_Rate(%)  R1_Q20  R2_Q20  R1_Q30  \
38    18P4063833M          27.31        92.69%  98.20%  95.94%  91.72%   
42    18P4063839M          32.00        93.71%  98.18%  95.91%  91.54%   
48    18P4063830M          24.08        92.50%  98.12%  95.17%  91.26%   
53    19P0126529M          69.64        92.71%  98.21%  95.43%  91.72%   
58    19P0126533M          60.09        88.71%  98.21%  95.48%  91.72%   
...           ...            ...           ...     ...     ...     ...   
1848  19P7247843M          37.42        93.36%  97.73%  96.36%  90.51%   
1849  19P7247847M          44.69        95.63%  98.07%  97.10%  91.30%   
1850  19P7247853M          43.26        95.02%  97.77%  96.98%  90.31%   
1851  19P7247861M          37.82        94.15%  97.78%  96.56%  90.67%   
1852  19P7247863M          35.73        93.61%  97.61%  96.48%  90.15%   

      R2_Q30 GC_Content BS_conversion_rate(lambda_DNA)  \
38    87.33%     27.24%

In [34]:
qc_raw_filt_subset2 = qc_raw_filt.loc[qc_raw_filt.loc[:, "sampleID_shift"].isin(library_raw_filt.loc[:, "建库编号"]), :]
qc_raw_filt_subset2.head

<bound method NDFrame.head of            Sample  Data_Size(Gb) Clean_Rate(%)  R1_Q20  R2_Q20  R1_Q30  \
24    18P4063823M          34.26        88.28%  97.83%  94.13%  91.06%   
28    18P4063845M          32.48        90.66%  97.61%  94.01%  90.27%   
38    18P4063833M          27.31        92.69%  98.20%  95.94%  91.72%   
42    18P4063839M          32.00        93.71%  98.18%  95.91%  91.54%   
48    18P4063830M          24.08        92.50%  98.12%  95.17%  91.26%   
...           ...            ...           ...     ...     ...     ...   
1848  19P7247843M          37.42        93.36%  97.73%  96.36%  90.51%   
1849  19P7247847M          44.69        95.63%  98.07%  97.10%  91.30%   
1850  19P7247853M          43.26        95.02%  97.77%  96.98%  90.31%   
1851  19P7247861M          37.82        94.15%  97.78%  96.56%  90.67%   
1852  19P7247863M          35.73        93.61%  97.61%  96.48%  90.15%   

      R2_Q30 GC_Content BS_conversion_rate(lambda_DNA)  \
24    83.80%     27.87%

In [35]:
qc_raw_filt_subset3 = qc_raw_filt_subset1.loc[qc_raw_filt_subset1.loc[:, "sampleID_shift"].isin(library_raw_filt.loc[:, "建库编号"]), :]
qc_raw_filt_subset3.head

<bound method NDFrame.head of            Sample  Data_Size(Gb) Clean_Rate(%)  R1_Q20  R2_Q20  R1_Q30  \
38    18P4063833M          27.31        92.69%  98.20%  95.94%  91.72%   
42    18P4063839M          32.00        93.71%  98.18%  95.91%  91.54%   
48    18P4063830M          24.08        92.50%  98.12%  95.17%  91.26%   
53    19P0126529M          69.64        92.71%  98.21%  95.43%  91.72%   
58    19P0126533M          60.09        88.71%  98.21%  95.48%  91.72%   
...           ...            ...           ...     ...     ...     ...   
1848  19P7247843M          37.42        93.36%  97.73%  96.36%  90.51%   
1849  19P7247847M          44.69        95.63%  98.07%  97.10%  91.30%   
1850  19P7247853M          43.26        95.02%  97.77%  96.98%  90.31%   
1851  19P7247861M          37.82        94.15%  97.78%  96.56%  90.67%   
1852  19P7247863M          35.73        93.61%  97.61%  96.48%  90.15%   

      R2_Q30 GC_Content BS_conversion_rate(lambda_DNA)  \
38    87.33%     27.24%

In [36]:
qc_raw_filt_subset4 = qc_raw_filt_subset3.loc[qc_raw_filt_subset3.loc[:, "测序文库名"].isin(pooling_raw_filt.loc[:, "测序文库名"]), :]
qc_raw_filt_subset4.head

<bound method NDFrame.head of            Sample  Data_Size(Gb) Clean_Rate(%)  R1_Q20  R2_Q20  R1_Q30  \
38    18P4063833M          27.31        92.69%  98.20%  95.94%  91.72%   
42    18P4063839M          32.00        93.71%  98.18%  95.91%  91.54%   
48    18P4063830M          24.08        92.50%  98.12%  95.17%  91.26%   
53    19P0126529M          69.64        92.71%  98.21%  95.43%  91.72%   
58    19P0126533M          60.09        88.71%  98.21%  95.48%  91.72%   
...           ...            ...           ...     ...     ...     ...   
1847  19P7247859M          38.99        94.28%  97.69%  96.85%  90.18%   
1848  19P7247843M          37.42        93.36%  97.73%  96.36%  90.51%   
1849  19P7247847M          44.69        95.63%  98.07%  97.10%  91.30%   
1850  19P7247853M          43.26        95.02%  97.77%  96.98%  90.31%   
1852  19P7247863M          35.73        93.61%  97.61%  96.48%  90.15%   

      R2_Q30 GC_Content BS_conversion_rate(lambda_DNA)  \
38    87.33%     27.24%

In [31]:
# idx1_2 = [x for x in list(qc_raw_filt_subset1.index) if x not in list(qc_raw_filt_subset2.index)]
# qc_raw_filt.loc[idx1_2, :]


In [32]:
with pd.ExcelWriter("final/qc.20200430.xlsx") as writer:
    qc_raw_filt.to_excel(writer, sheet_name="result", index=False)
    qc_raw_filt_subset4.to_excel(writer, sheet_name="filtBy_clinicalAndLibraryAndPooling", index=False)
seq_raw_filt.to_excel("final/sequence.20200430.xlsx", index=False)
pooling_raw_filt.to_excel("final/pooling.20200430.xlsx", index=False)
capture_raw_filt.to_excel("final/capture.20200430.xlsx", index=False)
library_raw_filt.to_excel("final/library.20200430.xlsx", index=False)
extract_raw_filt.to_excel("final/extract.20200430.xlsx", index=False)
clinical_raw_filt.to_excel("final/clinical.20200430.xlsx", index=False)